https://www.amazon.cn/gp/bestsellers/books

In [1]:
from IPython.display import HTML
import requests
r = requests.get('https://www.amazon.cn/gp/bestsellers/books')
open('amazonbook.html', 'w').write(r.text)
HTML('''
<iframe src="amazonbook.html" width=100% height=500px>
''')

This is a static page (frontpage). However, it uses "URI fragment" for pagination. i.e. the content after `#`. This piece of string is not sent to server or parsed by the server. It is analysed by browser so some javascript will trigger loading of more data. This is a typical case where you need to handle dynamic crawling.

Page 4 URL looks like this: https://www.amazon.cn/gp/bestsellers/books#4

In [2]:
r = requests.get('https://www.amazon.cn/gp/bestsellers/books#4')
open('amazonbook.html', 'w').write(r.text)
HTML('''
<iframe src="amazonbook.html" width=100% height=500px>
''')

You always see the first page because `requests.get` just ignore the parameter after `#`

In [3]:
import splinter

In [ ]:
from splinter import Browser
import time

browser = Browser()
# Visit URL
url = "https://www.amazon.cn/gp/bestsellers/books#4"
browser.visit(url)
# You need to wait for some time for the page to be fully loaded
time.sleep(2)

In [ ]:
#browser.html

In [ ]:
#open('amazonbook.html', 'w').write(browser.html)
#HTML('''
#<iframe src="amazonbook.html" width=100% height=500px>
#''')

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
mypage = BeautifulSoup(browser.html)

In [ ]:
items = []
for row in mypage.find_all('div', attrs={'class': 'zg_itemRow'}):
    item = {}
    item['No.'] = row.find('span', attrs={'class': 'zg_rankNumber'}).text.strip()
    item['title'] = row.find('div', attrs={'class': 'p13n-sc-truncated'}).text.strip()
    item['author'] = row.find('div', attrs={'class': 'a-row'}).text.strip()
    items.append(item)
df = pd.DataFrame(items)
df

In [ ]:
# Call the following line to quit browser.
# If you use the "with" statement, the browser will be automatically closed when leaving that scope
browser.quit()

In [ ]:
def scrape_one_page(url):
    items = []
    with Browser() as browser:
        # Visit URL
        #url = "https://www.amazon.cn/gp/bestsellers/books#4"
        browser.visit(url)
        # You need to wait for some time for the page to be fully loaded
        time.sleep(2)
        
        for row in mypage.find_all('div', attrs={'class': 'zg_itemRow'}):
            item = {}
            item['No.'] = row.find('span', attrs={'class': 'zg_rankNumber'}).text.strip()
            item['title'] = row.find('div', attrs={'class': 'p13n-sc-truncated'}).text.strip()
            item['author'] = row.find('div', attrs={'class': 'a-row'}).text.strip()
            items.append(item)
    return items


In [ ]:
all_page_items = []
for i in range(5):
    new_items = scrape_one_page('https://www.amazon.cn/gp/bestsellers/books#%s' % i)
    all_page_items.extend(new_items)

When you execute the above code, you will see your browser pop up 5 times!

In [ ]:
df = pd.DataFrame(all_page_items)
df

In [ ]:
len(df)